In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.gaussian_process.kernels import Matern, RBF, ConstantKernel as C, WhiteKernel, DotProduct

plt.rcParams.update({'figure.max_open_warning': 0})
warnings.filterwarnings("ignore")

In [2]:
# read data
features = pd.read_excel(r"/opt/jupyter_data/model/feature/final_features.xlsx")
target = pd.read_excel(r"/opt/jupyter_data/model/feature/final_data.xlsx")

In [3]:
# extract features and target data
X = features.values
y = target['logVDss'].values
# split the data, 80% is the training set and 20% is the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def train_model(model, param_grid):
    """This function trains the model and optimizes hyperparameters."""
    if len(param_grid) > 0:
        # setup grid search parameters
        gsearch = GridSearchCV(model,
                               param_grid,
                               cv=10,
                               scoring='neg_mean_squared_error',
                               verbose=1,
                               return_train_score=True)
        # search the grid
        gsearch.fit(X_train, y_train)
        # extract best model from the grid
        model = gsearch.best_estimator_
        best_idx = gsearch.best_index_
        # get cv scores for best model
        grid_results = pd.DataFrame(gsearch.cv_results_)
        cv_mean = abs(grid_results.loc[best_idx, 'mean_test_score'])
        cv_std = grid_results.loc[best_idx, 'std_test_score']
        print("Best parameters found by grid search are:", gsearch.best_params_)
    else:
        grid_results = []
        # model training
        model = model.fit(X_train, y_train)
        cv_results = cross_val_score(model,
                                     X_train,
                                     y_train,
                                     cv=10,
                                     scoring='neg_mean_squared_error')
        cv_mean = abs(np.mean(cv_results))
        cv_std = np.std(cv_results)
    # calculate the MSE and R2 of the train and test sets
    MSE_train_score = mean_squared_error(y_train, model.predict(X_train))
    MSE_test_score = mean_squared_error(y_test, model.predict(X_test))
    R2_train_score = model.score(X_train, y_train)
    R2_test_score = model.score(X_test, y_test)
    test_score = pd.Series({'R2': R2_test_score, 'MSE': MSE_test_score})

    # print stats on model performance
    print('----------------')
    print(model)
    print('----------------')
    print("GridSearchCV train R2:  ", R2_train_score)
    print("GridSearchCV train MSE:  ", MSE_train_score)
    print("GridSearchCV test R2:  ", R2_test_score)
    print("GridSearchCV test MSE:  ", MSE_test_score)
    print('cross_val: mean=', cv_mean, ', std=', cv_std)

    return model, test_score, grid_results

In [5]:
def fold_n(n, fold_error):
    # count the number of fold_error less than n
    number = sum[abs(fold_error["FE_score"]) < n]
    # calculate n-fold fold error
    n_fold_error = number / len(fold_error)

    return n_fold_error

In [6]:
def cross_validation(folds, lgb_reg):
    """This function gets the MSE, R2, MAE, RMSE values of the model's ten-fold cross-validation."""
    FE_score = []
    n_fold_error = []
    # ten-fold cross-validation
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    # record training and prediction MSE, R2, MAE, RMSE values
    score_dict = {'train_mse': [], 'test_mse': [], 'train_r2': [], 'test_r2': [], 'train_mae': [], 'test_mae': [],
                  'train_rmse': [], 'test_rmse': []}

    for i, (train_index, test_index) in enumerate(kf.split(X)):
        # split training and test set
        X_train_KFold, X_test_KFold = X[train_index], X[test_index]
        y_train_KFold, y_test_KFold = y[train_index], y[test_index]
        # train model
        lgb_reg.fit(X=X_train_KFold,
                    y=y_train_KFold)

        # make predictions
        y_train_KFold_predict = lgb_reg.predict(X_train_KFold)
        y_test_KFold_predict = lgb_reg.predict(X_test_KFold)

        # calculate train and test set MSE, R2, MAE, RMSE
        train_r2 = lgb_reg.score(X_train_KFold, y_train_KFold)
        test_r2 = lgb_reg.score(X_test_KFold, y_test_KFold)
        train_mse = mean_squared_error(y_train_KFold_predict, y_train_KFold)
        test_mse = mean_squared_error(y_test_KFold_predict, y_test_KFold)
        train_mae = mean_absolute_error(y_train_KFold_predict, y_train_KFold)
        test_mae = mean_absolute_error(y_test_KFold_predict, y_test_KFold)
        train_rmse = train_mse ** 0.5
        test_rmse = test_mse ** 0.5

        # calculate fold error
        for y_pred, y_exp in zip(y_test_KFold_predict, y_test_KFold):
            if y_pred > y_exp:
                z = y_pred / y_exp
            else:
                z = y_exp / y_pred
            FE_score.append(z)

        # merge training and prediction MSE, R2, MAE, RMSE values
        score_dict['train_mse'].append(train_mse)
        score_dict['test_mse'].append(test_mse)
        score_dict['train_r2'].append(train_r2)
        score_dict['test_r2'].append(test_r2)
        score_dict['train_mae'].append(train_mae)
        score_dict['test_mae'].append(test_mae)
        score_dict['train_rmse'].append(train_rmse)
        score_dict['test_rmse'].append(test_rmse)
    score_dict = pd.DataFrame(score_dict)

    FE_score = pd.DataFrame(FE_score, columns=["FE_score"])
    FE_score["FE_score"].astype(float)

    # calculate 2-, 3- and 4-fold error
    for m in (2, 3, 4):
        n_fold_error.append(fold_n(m, FE_score))

    score_dict.loc[0:2, 'fold_error'] = n_fold_error

    return score_dict

In [7]:
# places to store optimal models and scores
opt_models = dict()
model_name = "GPR"
opt_models[model_name], origin_score, origin_results = train_model(GPR(random_state=42), [])

----------------
GaussianProcessRegressor(random_state=42)
----------------
GridSearchCV train R2:   0.9967300917749176
GridSearchCV train MSE:   0.0012648497209242153
GridSearchCV test R2:   0.5965811225741517
GridSearchCV test MSE:   0.15850255035803199
cross_val: mean= 0.18060543737656543 , std= 0.03292183336093716


In [8]:
# hyperparameter optimization for kernel
opt_models1 = dict()

kernel_range = [1.0 * RBF(length_scale=1) + WhiteKernel(noise_level=1),
                Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=0.5),
                Matern(length_scale=0.484, nu=2.5) + WhiteKernel(noise_level=0.5),
                C(1.0, (1e-3, 1e3)) * RBF(10, (0.5, 2)),
                DotProduct() + WhiteKernel(noise_level=0.5)
                ]

opt_models1[model_name], score1, grid_results1 = train_model(GPR(random_state=42),
                                                             {'kernel': kernel_range}
                                                             )

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best parameters found by grid search are: {'kernel': 1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1)}
----------------
GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1),
                         random_state=42)
----------------
GridSearchCV train R2:   0.9948264784169097
GridSearchCV train MSE:   0.002001196021457805
GridSearchCV test R2:   0.7716157714569656
GridSearchCV test MSE:   0.08973175206030455
cross_val: mean= 0.09332549902140255 , std= 0.018961403304825764


In [9]:
# hyperparameter optimization for kernel
opt_models2 = dict()

kernel_range = [Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1e-5),
                Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=0.1),
                Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=0.5),
                Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1),
                ]

opt_models2[model_name], score2, grid_results2 = train_model(GPR(random_state=42),
                                                             {'kernel': kernel_range}
                                                             )

Fitting 10 folds for each of 4 candidates, totalling 40 fits
Best parameters found by grid search are: {'kernel': Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1e-05)}
----------------
GaussianProcessRegressor(kernel=Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1e-05),
                         random_state=42)
----------------
GridSearchCV train R2:   0.9950393417308228
GridSearchCV train MSE:   0.0019188572875653205
GridSearchCV test R2:   0.7845155735116527
GridSearchCV test MSE:   0.0846634430663668
cross_val: mean= 0.08735012574121062 , std= 0.020910345969207718


In [10]:
# hyperparameter optimization for kernel
opt_models3 = dict()

kernel_range = [Matern(length_scale=0.01, nu=1.5) + WhiteKernel(noise_level=1),
                Matern(length_scale=0.1, nu=1.5) + WhiteKernel(noise_level=1),
                Matern(length_scale=0.5, nu=1.5) + WhiteKernel(noise_level=1),
                Matern(length_scale=1, nu=1.5) + WhiteKernel(noise_level=1),
                ]

opt_models3[model_name], score3, grid_results3 = train_model(GPR(random_state=42),
                                                             {'kernel': kernel_range}
                                                             )

Fitting 10 folds for each of 4 candidates, totalling 40 fits
Best parameters found by grid search are: {'kernel': Matern(length_scale=1, nu=1.5) + WhiteKernel(noise_level=1)}
----------------
GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=1.5) + WhiteKernel(noise_level=1),
                         random_state=42)
----------------
GridSearchCV train R2:   0.9950393412418901
GridSearchCV train MSE:   0.0019188574766918405
GridSearchCV test R2:   0.7845155718470818
GridSearchCV test MSE:   0.08466344372037371
cross_val: mean= 0.0873501252332824 , std= 0.02091034648984532


In [11]:
# hyperparameter optimization for alpha
opt_models4 = dict()
opt_models4[model_name], score4, grid_results4 = train_model(
    GPR(kernel=Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1e-5), random_state=42),
    {'alpha': np.linspace(0, 0.001, 50)})

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best parameters found by grid search are: {'alpha': 4.0816326530612245e-05}
----------------
GaussianProcessRegressor(alpha=4.0816326530612245e-05,
                         kernel=Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1e-05),
                         random_state=42)
----------------
GridSearchCV train R2:   0.995039342524733
GridSearchCV train MSE:   0.0019188569804688901
GridSearchCV test R2:   0.7845155725663042
GridSearchCV test MSE:   0.08466344343779254
cross_val: mean= 0.08735012394679206 , std= 0.020910346799995688


In [12]:
# get the 10-fold cross validation score for the model
score = cross_validation(folds=10,
                         lgb_reg=GPR(kernel=Matern(length_scale=0.484, nu=1.5) + WhiteKernel(noise_level=1e-5),
                                     random_state=42))


In [13]:
# add score information before optimization and
# after hyperparameter optimization to the table
score.loc[0:1, 'origin_score'] = [origin_score['R2'], origin_score['MSE']]
score.loc[0:1, 'final_score'] = [score2['R2'], score2['MSE']]

In [14]:
# save file
score.to_excel("GP_score.xlsx", index=False)

In [15]:
# 10-fold cross validation mean of test set with respect to R2
score['test_r2'].mean()

0.8159164617805441